In [1]:
!pip install pyngrok
!pip install flask-cors
!ngrok config add-authtoken 2ws8hQMkwQmgCtaU4f4DJoqqwFl_7bK8uQ5o6bnVC5FpqNnBB

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [2]:
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from pyngrok import ngrok
import torch
from diffusers import StableDiffusionXLImg2ImgPipeline
from PIL import Image
import io
import os

# Init Flask
app = Flask(__name__)
CORS(app)

# Load the pipeline once
pipe = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

2025-06-28 13:51:52.701365: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751118712.915625      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751118712.974687      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/575 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.fp16.safete(…):   0%|          | 0.00/5.14G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.fp16.safeten(…):   0%|          | 0.00/167M [00:00<?, ?B/s]

vae_1_0/diffusion_pytorch_model.fp16.saf(…):   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
@app.route("/generate", methods=["POST"])
def generate():
    image_file = request.files["image"]
    prompt = request.form.get("prompt")
    strength = float(request.form.get("strength", 0.6))
    guidance_scale = float(request.form.get("guidance_scale", 15))
    num_inference_steps = int(request.form.get("num_inference_steps", 100))
    
    negative_prompt = (
        "blurry, low resolution, distorted, noisy, bad anatomy, "
        "poor lighting, jpeg artifacts, deformed, watermark, oversaturated"
    )

    init_image = Image.open(image_file.stream).convert("RGB").resize((1344, 1344))

    output = pipe(
        prompt=prompt,
        image=init_image,
        strength=strength,
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps,
        negative_prompt=negative_prompt
    )

    result_image = output.images[0]
    img_io = io.BytesIO()
    result_image.save(img_io, 'JPEG')
    img_io.seek(0)
    return send_file(img_io, mimetype='image/jpeg')

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print("Your ngrok URL:", public_url)

# Run Flask
app.run(port=5000)

Your ngrok URL: NgrokTunnel: "https://870a-35-224-246-46.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


  0%|          | 0/70 [00:00<?, ?it/s]